<a href="https://colab.research.google.com/github/jeromepatel/DL-University-Course-Practicals/blob/master/18BCE088_DL_practical_5_TL_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical 5 Transfer Learning on CIFAR10 using Xception Model

In [1]:
!nvidia-smi

Mon Mar 22 08:45:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Importing Libraries

In [2]:

#importing libraries
from keras.datasets.cifar10 import load_data
# from matplotlib import pyplot
from tensorflow import keras
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, BatchNormalization ,GlobalAveragePooling2D
from keras import backend as K
#from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

import numpy as np
# batch_size = 512
# num_classes = 10
# epochs = 30


In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
(X_train_data, Y_train), (X_test_data, Y_test) = load_data()
X_train_data.shape, X_test_data.shape

170500096/170498071 [==============================] - 11s 0us/step


((50000, 32, 32, 3), (10000, 32, 32, 3))

In [5]:
# X_train = X_train_data/255
# X_test = X_test_data/255

# del X_train_data, X_test_data

In [6]:
%%time
import cv2
rows = 71
columns  = 71
X_train_1 = np.array([cv2.resize(i, dsize=(rows, columns), interpolation=cv2.INTER_CUBIC) for i in X_train_data])
X_train = preprocess_input(X_train_1)
X_test_1 = np.array([cv2.resize(i, dsize=(rows, columns), interpolation=cv2.INTER_CUBIC) for i in X_test_data])
X_test = preprocess_input(X_test_1)

del X_train_data, X_test_data, X_train_1,X_test_1

num_classes = 10
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)
y_train,y_test = Y_train,Y_test
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, num_classes)
del y_test,y_train

CPU times: user 4.73 s, sys: 1.66 s, total: 6.38 s
Wall time: 6.97 s


In [7]:
X_train.shape

(50000, 71, 71, 3)

In [8]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(rows, columns, 3), pooling=None)

83689472/83683744 [==============================] - 2s 0us/step


In [9]:
for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 71, 71, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [10]:
%%time
from tensorflow.keras import layers
from tensorflow.keras import Model

#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(100,100,3),pooling = None)

# x_train,x_test = X_train,X_test
# X_train = base_model.predict(X_train)
# del x_train
# X_test = base_model.predict(X_test)
# del x_test


flat1 = Flatten()(base_model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
class2 = Dropout(0.4)(class1)
output = Dense(10, activation='softmax')(class2)
# define new model
model = Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer = Adam(lr = 0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])


training_history = None
batch_size = 128
epochs = 20
try:
   training_history =  model.fit(
    X_train,
        Y_train,
        validation_data = (X_test,Y_test),
        batch_size = batch_size,
        epochs = epochs,
        shuffle = True,
        verbose = 1
    )
except KeyboardInterrupt as e:
    print("\njust wait to complete training!!\n")

Epoch 1/20
391/391 [==============================] - 107s 190ms/step - loss: 1.8782 - acc: 0.6354 - val_loss: 0.7745 - val_acc: 0.7354
Epoch 2/20
391/391 [==============================] - 72s 184ms/step - loss: 0.7781 - acc: 0.7351 - val_loss: 0.7279 - val_acc: 0.7551
Epoch 3/20
391/391 [==============================] - 72s 185ms/step - loss: 0.7068 - acc: 0.7582 - val_loss: 0.7324 - val_acc: 0.7538
Epoch 4/20
391/391 [==============================] - 72s 185ms/step - loss: 0.6634 - acc: 0.7713 - val_loss: 0.7186 - val_acc: 0.7568
Epoch 5/20
391/391 [==============================] - 72s 185ms/step - loss: 0.6125 - acc: 0.7845 - val_loss: 0.7038 - val_acc: 0.7622
Epoch 6/20
391/391 [==============================] - 72s 185ms/step - loss: 0.5893 - acc: 0.7934 - val_loss: 0.7055 - val_acc: 0.7639
Epoch 7/20
391/391 [==============================] - 72s 185ms/step - loss: 0.5529 - acc: 0.8036 - val_loss: 0.7149 - val_acc: 0.7680
Epoch 8/20
391/391 [==============================] - 

In [11]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((50000, 71, 71, 3), (10000, 71, 71, 3), (50000, 10), (10000, 10))

In [12]:
#prediction using test set
prediction = model.predict(X_test)
y_test = np.argmax(Y_test, axis=1)
prediction = np.argmax(prediction, axis=1)
print('\nAccuracy:', accuracy_score(y_test, prediction, normalize=True))
print('\nClassification Report:' + classification_report(y_test, prediction) + '\nConfusion Matrix:')
print(confusion_matrix(y_test, prediction))


Accuracy: 0.7666

Classification Report:              precision    recall  f1-score   support

           0       0.78      0.79      0.79      1000
           1       0.82      0.88      0.85      1000
           2       0.77      0.69      0.73      1000
           3       0.63      0.63      0.63      1000
           4       0.74      0.68      0.71      1000
           5       0.68      0.76      0.72      1000
           6       0.73      0.78      0.75      1000
           7       0.82      0.78      0.80      1000
           8       0.84      0.86      0.85      1000
           9       0.87      0.81      0.84      1000

    accuracy                           0.77     10000
   macro avg       0.77      0.77      0.77     10000
weighted avg       0.77      0.77      0.77     10000

Confusion Matrix:
[[791  24  26  22  11   8  13   9  79  17]
 [ 17 878   0  10   0  15   2   0  17  61]
 [ 45   1 688  45  66  48  75  19  12   1]
 [ 17   9  37 630  29 173  68  24   7   6]
 [ 20   4 

In [ ]:
#@title Default title text
# model = Sequential()
# model.add(GlobalAveragePooling2D(input_shape = (1,1,2048)))
# model.add(Dropout(0.4))
# #model.add(Dense(256,input_dim = X_train.shape[1],activation='relu'))
# model.add(Dense(512,activation = 'relu'))
# model.add(Dropout(0.5))
# #model.add(BatchNormalization())
# # model.add(Dense(128,activation = 'relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(Y_test.shape[1], activation='softmax'))

In [20]:
#@title Default title text
# model = models.Sequential()
# model.add(GlobalAveragePooling2D(input_shape=(7, 7, 2048)))
# model.add(Dropout(0.2))
# model.add(Dense(Y_test.shape[1], activation='softmax'))

In [15]:
#@title Default title text
# model.compile(Adam(lr = 0.001),
#               loss = 'categorical_crossentropy',
#               metrics = ['accuracy'])

In [16]:
#@title Redundant Code { display-mode: "form" }
# %%time
# training_history = None
# batch_size = 128
# epochs = 20
# try:
#    training_history =  model.fit(
#     X_train,
#         Y_train,
#         validation_data = (X_test,Y_test),
#         batch_size = batch_size,
#         epochs = epochs,
#         shuffle = True,
#         verbose = 1
#     )
# except KeyboardInterrupt as e:
#     print("\njust wait to complete training!!\n")

In [17]:
#@title Default title text
# X_train = X_train/255   #in range (0,1) now 
# X_test = X_test/255
# print(X_train[0].shape)
# Y_train[0]

In [18]:
#@title Default title text
# img_rows, img_cols = 32, 32
# y_train,y_test = Y_train,Y_test
# input_shape = (img_rows, img_cols, 3)
# # convert class vectors to binary class matrices
# Y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
# Y_test = tf.keras.utils.to_categorical(Y_test, num_classes)

In [19]:
#@title redundant Code { display-mode: "form" }
# import cv2
# from numpy import array, argmax
# from sklearn import metrics
# import matplotlib.pyplot as plt
# from tensorflow.keras import datasets, utils, models, optimizers
# from tensorflow.keras.applications.xception import Xception, preprocess_input
# from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
# (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# # print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# x_train = array([cv2.resize(i, dsize=(72, 72), interpolation=cv2.INTER_CUBIC) for i in x_train])
# x_train = preprocess_input(x_train)
# x_test = array([cv2.resize(i, dsize=(72, 72), interpolation=cv2.INTER_CUBIC) for i in x_test])
# x_test = preprocess_input(x_test)

# y_train = utils.to_categorical(y_train)
# y_test = utils.to_categorical(y_test)
# model_xception = Xception(include_top=False, weights='imagenet', input_shape=(72, 72, 3), pooling=None)

# del y_train,y_test
# #model_xception.summary()

# x_train = model_xception.predict(x_train)
# x_test = model_xception.predict(x_test)
# model = models.Sequential()
# model.add(GloalAveragePooling2D(input_size=(7, 7, 2048)))
# model.add(Dropout(0.3))
# model.add(Dense(x_test.shape[1], activaton='softmax'))

# # model.summary()
# model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=10, verbose = True)
# prediction = model.predict(x_test)

# print(prediction)

# y_test = argmax(y_test, axis=1)
# prediction = argmax(prediction, axis=1)

# print('\nAccuracy:', metrics.accuracy_score(y_test, prediction, normalize=True))
# print('\nClassification Report:' + metrics.classification_report(y_test, prediction) + '\nConfusion Matrix:')
# print(metrics.confusion_matrix(y_test, prediction))